# Automated Features


In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from scipy.sparse import hstack
from xgboost.sklearn import XGBClassifier # <3
from sklearn.model_selection import train_test_split
import gc
import matplotlib.pyplot as plt

from scipy.sparse.csr import csr_matrix

pd.options.mode.chained_assignment = None


In [2]:
import featuretools as ft

In [3]:

pd.set_option('display.width', 400)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 200)


In [4]:
df = pd.read_csv('events_up_to_01062018.csv', low_memory = False)
df_labels= pd.read_csv('labels_training_set.csv', low_memory = False)


In [5]:
df.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,search_term,staticpage,campaign_source,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df['session_id'] = df.index

In [15]:
df.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,search_term,staticpage,campaign_source,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,session_id
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4


In [8]:
df_person = df['person'].drop_duplicates().to_frame()

In [29]:
df['session']=df.index

In [28]:
es = ft.EntitySet(id = 'person')
es = es.entity_from_dataframe(entity_id = 'person_id', dataframe = df_person, index = 'person')

In [31]:

df.columns

Index(['timestamp', 'event', 'person', 'url', 'sku', 'model', 'condition', 'storage', 'color', 'skus', 'search_term', 'staticpage', 'campaign_source', 'search_engine', 'channel', 'new_vs_returning', 'city', 'region', 'country', 'device_type', 'screen_resolution', 'operating_system_version', 'browser_version', 'session_id', 'session'], dtype='object')

In [36]:
es = es.entity_from_dataframe(entity_id = 'session', dataframe = df, 
                            index = 'session_id', time_index = 'timestamp')


In [37]:
es

Entityset: person
  Entities:
    person_id [Rows: 38829, Columns: 1]
    session [Rows: 2341681, Columns: 25]
  Relationships:
    No relationships

In [39]:
# Relationship between clients and previous loans
r_client_previous = ft.Relationship(es['person_id']['person'],
                                    es['session']['person'])

# Add the relationship to the entity set
es = es.add_relationship(r_client_previous)

2018-11-12 09:27:36,172 featuretools.entityset - WARNING    Not adding duplicate relationship: <Relationship: session.person -> person_id.person>


In [40]:
es

Entityset: person
  Entities:
    person_id [Rows: 38829, Columns: 1]
    session [Rows: 2341681, Columns: 25]
  Relationships:
    session.person -> person_id.person

In [43]:
features, feature_names = ft.dfs(entityset = es, target_entity = 'person_id', 
                                 agg_primitives = ['mean', 'max', 'percent_true', 'last'],
trans_primitives = ['years', 'month', 'subtract', 'divide'])

In [45]:
features

,MEAN(session.sku),MEAN(session.session),MAX(session.sku),MAX(session.session),LAST(session.event),LAST(session.url),LAST(session.sku),LAST(session.model),LAST(session.condition),LAST(session.storage),LAST(session.color),LAST(session.search_term),LAST(session.staticpage),LAST(session.campaign_source),LAST(session.search_engine),LAST(session.channel),LAST(session.new_vs_returning),LAST(session.city),LAST(session.region),LAST(session.country),LAST(session.device_type),LAST(session.screen_resolution),LAST(session.operating_system_version),LAST(session.browser_version),LAST(session.session),...,MEAN(session.session) / LAST(session.session),MEAN(session.session) / LAST(session.sku),MEAN(session.sku) / MEAN(session.session),LAST(session.session) / MAX(session.session),LAST(session.sku) / MAX(session.session),LAST(session.session) / MAX(session.sku),LAST(session.sku) / MAX(session.sku),MEAN(session.sku) / MAX(session.session),LAST(session.sku) / LAST(session.session),MEAN(session.sku) / MAX(session.sku),MAX(session.sku) / LAST(session.sku),MAX(session.session) / LAST(session.sku),LAST(session.session) / LAST(session.sku),LAST(session.sku) / MEAN(session.session),MEAN(session.session) / MAX(session.sku),MEAN(session.session) / MAX(session.session),MAX(session.session) / MEAN(session.session),MAX(session.session) / LAST(session.session),MAX(session.sku) / MEAN(session.session),MAX(session.session) / MAX(session.sku),LAST(session.session) / MEAN(session.session),LAST(session.sku) / MEAN(session.sku),MAX(session.sku) / LAST(session.session),MEAN(session.sku) / LAST(session.sku),MAX(session.sku) / MAX(session.session)
person,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0008ed71,6374.666667,1.885993e+06,8247.0,2336761,checkout,NaN,7505.0,LG G4 H818P,Bom,32GB,Preto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1505383,...,1.252833,251.298179,0.003380,0.644218,0.003212,182.537044,0.910028,0.002728,0.004985,0.772968,1.098867,311.360560,200.584011,0.003979,228.688351,0.807097,1.239008,1.552270,0.004373,283.346793,0.798191,1.177316,0.005478,0.849389,0.003529
00091926,7190.058824,3.929293e+05,15819.0,2146953,brand listing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1777801,...,0.221020,NaN,0.018299,0.828058,NaN,112.383905,NaN,0.003349,NaN,0.454520,NaN,NaN,NaN,NaN,24.839074,0.183017,5.463968,1.207645,0.040259,135.719894,4.524481,NaN,0.008898,NaN,0.007368
00091a7a,5487.666667,1.430326e+06,8261.0,2219186,brand listing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1912556,...,0.747861,NaN,0.003837,0.861828,NaN,231.516281,NaN,0.002473,NaN,0.664286,NaN,NaN,NaN,NaN,173.142017,0.644527,1.551524,1.160325,0.005776,268.634064,1.337147,NaN,0.004319,NaN,0.003723
000ba417,6559.893750,1.068300e+06,15759.0,2229932,conversion,NaN,7631.0,Motorola Moto G3 4G,Bom,16GB,Preto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,945641,...,1.129710,139.994725,0.006140,0.424067,0.003422,60.006409,0.484231,0.002942,0.008070,0.416263,2.065129,292.220155,123.920980,0.007143,67.789818,0.479073,2.087365,2.358117,0.014751,141.502126,0.885183,1.163281,0.016665,0.859637,0.007067
000c79fe,9944.000000,2.896774e+05,9944.0,2174099,searched products,NaN,NaN,NaN,NaN,NaN,NaN,Galaxy s8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62823,...,4.611008,NaN,0.034328,0.028896,NaN,6.317679,NaN,0.004574,NaN,1.000000,NaN,NaN,NaN,NaN,29.130868,0.133240,7.505243,34.606736,0.034328,218.634252,0.216872,NaN,0.158286,NaN,0.004574
000e4d9e,4320.797059,3.118005e+05,15730.0,2173028,visited site,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Referral,Returning,Igarape,Minas Gerais,Brazil,Computer,1536x864,Windows 10,Chrome 67.0,2173028,...,0.143487,NaN,0.013858,1.000000,NaN,138.145455,NaN,0.001988,NaN,0.274685,NaN,NaN,NaN,NaN,19.822026,0.143487,6.969290,1.000000,0.050449,138.145455,6.969290,NaN,0.007239,NaN,0.007239
000e619d,6904.172414,1.233865e+06,13404.0,2259744,visited site,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Paid,Returning,São

In [47]:
features, feature_names = ft.dfs(entityset=es, target_entity='person_id', 
                                 max_depth = 2)

features.head()

,SUM(session.sku),SUM(session.session),STD(session.sku),STD(session.session),MAX(session.sku),MAX(session.session),SKEW(session.sku),SKEW(session.session),MIN(session.sku),MIN(session.session),MEAN(session.sku),MEAN(session.session),COUNT(session),NUM_UNIQUE(session.event),NUM_UNIQUE(session.url),NUM_UNIQUE(session.model),NUM_UNIQUE(session.condition),NUM_UNIQUE(session.storage),NUM_UNIQUE(session.color),NUM_UNIQUE(session.search_term),NUM_UNIQUE(session.staticpage),NUM_UNIQUE(session.campaign_source),NUM_UNIQUE(session.search_engine),NUM_UNIQUE(session.channel),NUM_UNIQUE(session.new_vs_returning),...,MODE(session.country),MODE(session.device_type),MODE(session.screen_resolution),MODE(session.operating_system_version),MODE(session.browser_version),SUM(session.NUMWORDS(skus)),SUM(session.CHARACTERS(skus)),STD(session.NUMWORDS(skus)),STD(session.CHARACTERS(skus)),MAX(session.NUMWORDS(skus)),MAX(session.CHARACTERS(skus)),SKEW(session.NUMWORDS(skus)),SKEW(session.CHARACTERS(skus)),MIN(session.NUMWORDS(skus)),MIN(session.CHARACTERS(skus)),MEAN(session.NUMWORDS(skus)),MEAN(session.CHARACTERS(skus)),NUM_UNIQUE(session.DAY(timestamp)),NUM_UNIQUE(session.YEAR(timestamp)),NUM_UNIQUE(session.MONTH(timestamp)),NUM_UNIQUE(session.WEEKDAY(timestamp)),MODE(session.DAY(timestamp)),MODE(session.YEAR(timestamp)),MODE(session.MONTH(timestamp)),MODE(session.WEEKDAY(timestamp))
person,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0008ed71,19124.0,11315957,2626.717787,422725.090373,8247.0,2336761,-1.577862,0.130100,3372.0,1505383,6374.666667,1.885993e+06,6,3,0,3,2,2,3,0,0,0,0,1,2,...,Brazil,Computer,1920x1080,Windows 10,Chrome 66.0,6,59,0.0,24.086649,1,59,0.0,2.449490,1,0,1,9.833333,1,1,1,1,17,2018,5,3
00091926,2689082.0,176032329,2983.039678,628799.773981,15819.0,2146953,-0.245658,2.175637,1365.0,129,7190.058824,3.929293e+05,448,5,11,36,4,5,20,0,0,3,0,3,2,...,Brazil,Computer,1024x768,Windows 7,Chrome 66.0,448,971,0.0,10.225034,1,60,0.0,5.052546,1,0,1,2.167411,22,1,1,7,13,2018,5,6
00091a7a,16463.0,14303262,3646.651798,694823.466211,8261.0,2219186,-1.442907,-0.412550,1357.0,630080,5487.666667,1.430326e+06,10,4,1,3,2,3,3,0,0,1,0,1,1,...,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 43.0,10,286,0.0,30.148521,1,58,0.0,0.000350,1,0,1,28.600000,1,1,1,1,26,2018,3,0
000ba417,1049583.0,220069748,3659.929565,498486.409687,15759.0,2229932,0.640588,0.837486,220.0,244746,6559.893750,1.068300e+06,206,8,1,26,3,6,20,0,0,1,1,1,2,...,Brazil,Computer,1024x768,Windows 10,Chrome 66.0,206,2263,0.0,23.337498,1,63,0.0,1.673444,1,0,1,10.985437,3,1,1,2,24,2018,5,3
000c79fe,39776.0,4924515,0.000000,647464.281020,9944.0,2174099,0.000000,2.654337,9944.0,57876,9944.000000,2.896774e+05,17,7,1,1,1,1,1,7,0,1,1,1,1,...,Brazil,Smartphone,360x640,Android 7,Chrome Mobile 66.0,17,593,0.0,30.530481,1,66,0.0,-0.311727,1,0,1,34.882353,1,1,1,1,29,2018,5,1


In [64]:
df_not_numerical = features.select_dtypes(['number'])
df_not_numerical.head()

,SUM(session.sku),SUM(session.session),STD(session.sku),STD(session.session),MAX(session.sku),MAX(session.session),SKEW(session.sku),SKEW(session.session),MIN(session.sku),MIN(session.session),MEAN(session.sku),MEAN(session.session),COUNT(session),NUM_UNIQUE(session.event),NUM_UNIQUE(session.url),NUM_UNIQUE(session.model),NUM_UNIQUE(session.condition),NUM_UNIQUE(session.storage),NUM_UNIQUE(session.color),NUM_UNIQUE(session.search_term),NUM_UNIQUE(session.staticpage),NUM_UNIQUE(session.campaign_source),NUM_UNIQUE(session.search_engine),NUM_UNIQUE(session.channel),NUM_UNIQUE(session.new_vs_returning),...,NUM_UNIQUE(session.country),NUM_UNIQUE(session.device_type),NUM_UNIQUE(session.screen_resolution),NUM_UNIQUE(session.operating_system_version),NUM_UNIQUE(session.browser_version),SUM(session.NUMWORDS(skus)),SUM(session.CHARACTERS(skus)),STD(session.NUMWORDS(skus)),STD(session.CHARACTERS(skus)),MAX(session.NUMWORDS(skus)),MAX(session.CHARACTERS(skus)),SKEW(session.NUMWORDS(skus)),SKEW(session.CHARACTERS(skus)),MIN(session.NUMWORDS(skus)),MIN(session.CHARACTERS(skus)),MEAN(session.NUMWORDS(skus)),MEAN(session.CHARACTERS(skus)),NUM_UNIQUE(session.DAY(timestamp)),NUM_UNIQUE(session.YEAR(timestamp)),NUM_UNIQUE(session.MONTH(timestamp)),NUM_UNIQUE(session.WEEKDAY(timestamp)),MODE(session.DAY(timestamp)),MODE(session.YEAR(timestamp)),MODE(session.MONTH(timestamp)),MODE(session.WEEKDAY(timestamp))
person,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0008ed71,19124.0,11315957,2626.717787,422725.090373,8247.0,2336761,-1.577862,0.130100,3372.0,1505383,6374.666667,1.885993e+06,6,3,0,3,2,2,3,0,0,0,0,1,2,...,1,1,1,1,1,6,59,0.0,24.086649,1,59,0.0,2.449490,1,0,1,9.833333,1,1,1,1,17,2018,5,3
00091926,2689082.0,176032329,2983.039678,628799.773981,15819.0,2146953,-0.245658,2.175637,1365.0,129,7190.058824,3.929293e+05,448,5,11,36,4,5,20,0,0,3,0,3,2,...,1,1,1,1,1,448,971,0.0,10.225034,1,60,0.0,5.052546,1,0,1,2.167411,22,1,1,7,13,2018,5,6
00091a7a,16463.0,14303262,3646.651798,694823.466211,8261.0,2219186,-1.442907,-0.412550,1357.0,630080,5487.666667,1.430326e+06,10,4,1,3,2,3,3,0,0,1,0,1,1,...,1,1,1,1,1,10,286,0.0,30.148521,1,58,0.0,0.000350,1,0,1,28.600000,1,1,1,1,26,2018,3,0
000ba417,1049583.0,220069748,3659.929565,498486.409687,15759.0,2229932,0.640588,0.837486,220.0,244746,6559.893750,1.068300e+06,206,8,1,26,3,6,20,0,0,1,1,1,2,...,1,1,1,1,1,206,2263,0.0,23.337498,1,63,0.0,1.673444,1,0,1,10.985437,3,1,1,2,24,2018,5,3
000c79fe,39776.0,4924515,0.000000,647464.281020,9944.0,2174099,0.000000,2.654337,9944.0,57876,9944.000000,2.896774e+05,17,7,1,1,1,1,1,7,0,1,1,1,1,...,1,1,1,1,1,17,593,0.0,30.530481,1,66,0.0,-0.311727,1,0,1,34.882353,1,1,1,1,29,2018,5,1


## Diff de lo de caro 0.62

In [128]:
df_diff = pd.read_csv('diff.csv')
df_not_numerical = df_not_numerical.merge(df_diff , left_on=df_not_numerical.index, right_on='person', how='left')


ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat

In [129]:
df_not_numerical.head()

,SUM(session.sku),SUM(session.session),STD(session.sku),STD(session.session),MAX(session.sku),MAX(session.session),SKEW(session.sku),SKEW(session.session),MIN(session.sku),MIN(session.session),MEAN(session.sku),MEAN(session.session),COUNT(session),NUM_UNIQUE(session.event),NUM_UNIQUE(session.url),NUM_UNIQUE(session.model),NUM_UNIQUE(session.condition),NUM_UNIQUE(session.storage),NUM_UNIQUE(session.color),NUM_UNIQUE(session.search_term),NUM_UNIQUE(session.staticpage),NUM_UNIQUE(session.campaign_source),NUM_UNIQUE(session.search_engine),NUM_UNIQUE(session.channel),NUM_UNIQUE(session.new_vs_returning),...,NUM_UNIQUE(session.browser_version),SUM(session.NUMWORDS(skus)),SUM(session.CHARACTERS(skus)),STD(session.NUMWORDS(skus)),STD(session.CHARACTERS(skus)),MAX(session.NUMWORDS(skus)),MAX(session.CHARACTERS(skus)),SKEW(session.NUMWORDS(skus)),SKEW(session.CHARACTERS(skus)),MIN(session.NUMWORDS(skus)),MIN(session.CHARACTERS(skus)),MEAN(session.NUMWORDS(skus)),MEAN(session.CHARACTERS(skus)),NUM_UNIQUE(session.DAY(timestamp)),NUM_UNIQUE(session.YEAR(timestamp)),NUM_UNIQUE(session.MONTH(timestamp)),NUM_UNIQUE(session.WEEKDAY(timestamp)),MODE(session.DAY(timestamp)),MODE(session.YEAR(timestamp)),MODE(session.MONTH(timestamp)),MODE(session.WEEKDAY(timestamp)),person,diff_mean,diff_max,session_count
0,19124.0,11315957,2626.717787,422725.090373,8247.0,2336761,-1.577862,0.130100,3372.0,1505383,6374.666667,1.885993e+06,6,3,0,3,2,2,3,0,0,0,0,1,2,...,1,6,59,0.0,24.086649,1,59,0.0,2.449490,1,0,1,9.833333,1,1,1,1,17,2018,5,3,0008ed71,0.001389,0.001389,1.0
1,2689082.0,176032329,2983.039678,628799.773981,15819.0,2146953,-0.245658,2.175637,1365.0,129,7190.058824,3.929293e+05,448,5,11,36,4,5,20,0,0,3,0,3,2,...,1,448,971,0.0,10.225034,1,60,0.0,5.052546,1,0,1,2.167411,22,1,1,7,13,2018,5,6,00091926,0.142760,0.501667,31.0
2,16463.0,14303262,3646.651798,694823.466211,8261.0,2219186,-1.442907,-0.412550,1357.0,630080,5487.666667,1.430326e+06,10,4,1,3,2,3,3,0,0,1,0,1,1,...,1,10,286,0.0,30.148521,1,58,0.0,0.000350,1,0,1,28.600000,1,1,1,1,26,2018,3,0,00091a7a,0.171111,0.171111,1.0
3,1049583.0,220069748,3659.929565,498486.409687,15759.0,2229932,0.640588,0.837486,220.0,244746,6559.893750,1.068300e+06,206,8,1,26,3,6,20,0,0,1,1,1,2,...,1,206,2263,0.0,23.337498,1,63,0.0,1.673444,1,0,1,10.985437,3,1,1,2,24,2018,5,3,000ba417,0.128333,0.464444,13.0
4,39776.0,4924515,0.000000,647464.281020,9944.0,2174099,0.000000,2.654337,9944.0,57876,9944.000000,2.896774e+05,17,7,1,1,1,1,1,7,0,1,1,1,1,...,1,17,593,0.0,30.530481,1,66,0.0,-0.311727,1,0,1,34.882353,1,1,1,1,29,2018,5,1,000c79fe,0.184444,0.333889,3.0


## XGboost

In [135]:
df_train = df_labels.merge(df_not_numerical , left_on='person', right_on='person' , how='inner')

In [136]:
df_train.shape

(19414, 57)

In [137]:
df_train.head()

,person,label,SUM(session.sku),SUM(session.session),STD(session.sku),STD(session.session),MAX(session.sku),MAX(session.session),SKEW(session.sku),SKEW(session.session),MIN(session.sku),MIN(session.session),MEAN(session.sku),MEAN(session.session),COUNT(session),NUM_UNIQUE(session.event),NUM_UNIQUE(session.url),NUM_UNIQUE(session.model),NUM_UNIQUE(session.condition),NUM_UNIQUE(session.storage),NUM_UNIQUE(session.color),NUM_UNIQUE(session.search_term),NUM_UNIQUE(session.staticpage),NUM_UNIQUE(session.campaign_source),NUM_UNIQUE(session.search_engine),...,NUM_UNIQUE(session.operating_system_version),NUM_UNIQUE(session.browser_version),SUM(session.NUMWORDS(skus)),SUM(session.CHARACTERS(skus)),STD(session.NUMWORDS(skus)),STD(session.CHARACTERS(skus)),MAX(session.NUMWORDS(skus)),MAX(session.CHARACTERS(skus)),SKEW(session.NUMWORDS(skus)),SKEW(session.CHARACTERS(skus)),MIN(session.NUMWORDS(skus)),MIN(session.CHARACTERS(skus)),MEAN(session.NUMWORDS(skus)),MEAN(session.CHARACTERS(skus)),NUM_UNIQUE(session.DAY(timestamp)),NUM_UNIQUE(session.YEAR(timestamp)),NUM_UNIQUE(session.MONTH(timestamp)),NUM_UNIQUE(session.WEEKDAY(timestamp)),MODE(session.DAY(timestamp)),MODE(session.YEAR(timestamp)),MODE(session.MONTH(timestamp)),MODE(session.WEEKDAY(timestamp)),diff_mean,diff_max,session_count
0,0566e9c1,0,115737.0,115089045,3059.600084,4.995813e+05,10519.0,2310292,0.725500,-1.029633,87.0,443335,4629.480000,1.692486e+06,68,9,1,12,4,3,6,0,1,1,1,...,2,1,68,1222,0.0,28.051110,1,62,0.0,0.925216,1,0,1,17.970588,10,1,1,7,22,2018,5,1,0.059618,0.247778,8.0
1,6ec7ee77,0,0.0,2726888,NaN,1.175104e+06,NaN,2194368,NaN,NaN,NaN,532520,NaN,1.363444e+06,2,2,1,0,0,0,0,0,0,1,0,...,1,1,2,0,0.0,0.000000,1,0,NaN,NaN,1,0,1,0.000000,1,1,1,1,28,2018,5,0,NaN,NaN,NaN
2,abe7a2fb,0,109892.0,163546011,2383.991959,4.496767e+05,9412.0,2291893,1.602912,0.127437,1301.0,1076396,3434.125000,1.703604e+06,96,8,6,7,3,4,7,2,0,3,1,...,1,3,96,1656,0.0,26.682145,1,68,0.0,0.974114,1,0,1,17.250000,11,1,4,5,29,2018,4,1,0.095119,0.346944,21.0
3,34728364,0,205447.0,46219668,3123.159931,5.768992e+05,13277.0,2258338,-0.382143,0.843358,3107.0,793404,8560.291667,1.249180e+06,37,5,0,11,3,3,5,0,0,0,1,...,1,1,37,444,0.0,24.063573,1,63,0.0,1.603400,1,0,1,12.000000,4,1,1,3,25,2018,5,4,0.043148,0.080556,3.0
4,87ed62de,0,70491.0,20648365,8.684364,2.784151e+05,6413.0,2290359,-2.403223,4.066189,6385.0,1140230,6408.272727,1.214610e+06,17,4,1,1,2,1,3,0,0,2,0,...,1,1,17,0,0.0,0.000000,1,0,0.0,0.000000,1,0,1,0.000000,2,1,1,2,18,2018,5,4,0.155278,0.172222,2.0


In [138]:
X, y = df_train.iloc[:,2:],df_train.iloc[:,1]
X = X.fillna(0)
X.head()

,SUM(session.sku),SUM(session.session),STD(session.sku),STD(session.session),MAX(session.sku),MAX(session.session),SKEW(session.sku),SKEW(session.session),MIN(session.sku),MIN(session.session),MEAN(session.sku),MEAN(session.session),COUNT(session),NUM_UNIQUE(session.event),NUM_UNIQUE(session.url),NUM_UNIQUE(session.model),NUM_UNIQUE(session.condition),NUM_UNIQUE(session.storage),NUM_UNIQUE(session.color),NUM_UNIQUE(session.search_term),NUM_UNIQUE(session.staticpage),NUM_UNIQUE(session.campaign_source),NUM_UNIQUE(session.search_engine),NUM_UNIQUE(session.channel),NUM_UNIQUE(session.new_vs_returning),...,NUM_UNIQUE(session.operating_system_version),NUM_UNIQUE(session.browser_version),SUM(session.NUMWORDS(skus)),SUM(session.CHARACTERS(skus)),STD(session.NUMWORDS(skus)),STD(session.CHARACTERS(skus)),MAX(session.NUMWORDS(skus)),MAX(session.CHARACTERS(skus)),SKEW(session.NUMWORDS(skus)),SKEW(session.CHARACTERS(skus)),MIN(session.NUMWORDS(skus)),MIN(session.CHARACTERS(skus)),MEAN(session.NUMWORDS(skus)),MEAN(session.CHARACTERS(skus)),NUM_UNIQUE(session.DAY(timestamp)),NUM_UNIQUE(session.YEAR(timestamp)),NUM_UNIQUE(session.MONTH(timestamp)),NUM_UNIQUE(session.WEEKDAY(timestamp)),MODE(session.DAY(timestamp)),MODE(session.YEAR(timestamp)),MODE(session.MONTH(timestamp)),MODE(session.WEEKDAY(timestamp)),diff_mean,diff_max,session_count
0,115737.0,115089045,3059.600084,4.995813e+05,10519.0,2310292,0.725500,-1.029633,87.0,443335,4629.480000,1.692486e+06,68,9,1,12,4,3,6,0,1,1,1,3,2,...,2,1,68,1222,0.0,28.051110,1,62,0.0,0.925216,1,0,1,17.970588,10,1,1,7,22,2018,5,1,0.059618,0.247778,8.0
1,0.0,2726888,0.000000,1.175104e+06,0.0,2194368,0.000000,0.000000,0.0,532520,0.000000,1.363444e+06,2,2,1,0,0,0,0,0,0,1,0,1,1,...,1,1,2,0,0.0,0.000000,1,0,0.0,0.000000,1,0,1,0.000000,1,1,1,1,28,2018,5,0,0.000000,0.000000,0.0
2,109892.0,163546011,2383.991959,4.496767e+05,9412.0,2291893,1.602912,0.127437,1301.0,1076396,3434.125000,1.703604e+06,96,8,6,7,3,4,7,2,0,3,1,5,2,...,1,3,96,1656,0.0,26.682145,1,68,0.0,0.974114,1,0,1,17.250000,11,1,4,5,29,2018,4,1,0.095119,0.346944,21.0
3,205447.0,46219668,3123.159931,5.768992e+05,13277.0,2258338,-0.382143,0.843358,3107.0,793404,8560.291667,1.249180e+06,37,5,0,11,3,3,5,0,0,0,1,2,2,...,1,1,37,444,0.0,24.063573,1,63,0.0,1.603400,1,0,1,12.000000,4,1,1,3,25,2018,5,4,0.043148,0.080556,3.0
4,70491.0,20648365,8.684364,2.784151e+05,6413.0,2290359,-2.403223,4.066189,6385.0,1140230,6408.272727,1.214610e+06,17,4,1,1,2,1,3,0,0,2,0,1,1,...,1,1,17,0,0.0,0.000000,1,0,0.0,0.000000,1,0,1,0.000000,2,1,1,2,18,2018,5,4,0.155278,0.172222,2.0


In [139]:
import xgboost as xgb
from sklearn.linear_model import LinearRegression

In [140]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [141]:
reg = LinearRegression().fit(X, y)

In [142]:
persons = df_labels['person']
df_predict = df_not_numerical.loc[~df_not_numerical.person.isin(persons)]
ppl_to_predict = (df.loc[~df['person'].isin(persons)])['person'].to_frame()
ppl_to_predict = ppl_to_predict.drop_duplicates('person')
df_predict = ppl_to_predict.merge(df_predict, left_on = 'person' , right_on = 'person', how='left')
X_predict = df_predict.drop(['person'], axis=1)

In [143]:
X_predict = X_predict.fillna(0)

In [144]:
entrie = reg.predict(X_predict)

In [145]:
seriesita = pd.Series(entrie)

In [146]:
df_entrie = df_predict['person'].to_frame()
df_entrie['label'] = seriesita

In [147]:
df_entrie.head()

,person,label
0,4886f805,0.032995
1,0297fc1e,-0.008910
2,2d681dd8,0.011157
3,cccea85e,0.126387
4,4c8a8b93,0.048702


In [148]:
df_entrie = df_entrie.fillna(0)

In [149]:
num = df_entrie._get_numeric_data()
num[num < 0] = 0

In [150]:
df_entrie.to_csv(path_or_buf = 'automated.csv', index = False)

In [151]:
df_entrie.shape

(19415, 2)

In [152]:
df_entrie['label'].nlargest()

4471    0.487392
4153    0.416594
433     0.411209
1642    0.397671
4152    0.388914
Name: label, dtype: float64

## Scoring

Para evaluar usen esta medida que me da valores muy parecidos a los de kaggle, para hacer las predicciones usen el otro

In [153]:
my_classifier1 = xgb.XGBClassifier(objective ='reg:linear', 
                colsample_bytree = 1, learning_rate = 0.1,
                max_depth = 6,
                subsample = 0.8,
                gamma = 1,
                n_estimators = 10)

In [154]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [158]:
my_classifier1.fit(X_train,y_train)
entrie = my_classifier1.predict_proba(X_predict)[:,1]

In [159]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,  my_classifier1.predict_proba(X_test)[:,1])

0.8037712784965938

In [160]:
entrie

array([0.19248766, 0.18188411, 0.19248766, ..., 0.21079084, 0.26255932,
       0.26255932], dtype=float32)

In [161]:
seriesita = pd.Series(entrie)

In [162]:
df_entrie = df_predict['person'].to_frame()
df_entrie['label'] = seriesita

In [163]:
df_entrie.head()

,person,label
0,4886f805,0.192488
1,0297fc1e,0.181884
2,2d681dd8,0.192488
3,cccea85e,0.211138
4,4c8a8b93,0.196260


In [164]:
df_entrie = df_entrie.fillna(0)

In [165]:
num = df_entrie._get_numeric_data()
num[num < 0] = 0

In [166]:
df_entrie.to_csv(path_or_buf = 'automated.csv', index = False)

In [167]:
df_entrie.shape

(19415, 2)

In [168]:
df_entrie['label'].nlargest()

19324    0.674144
13144    0.672369
19323    0.668625
17535    0.615890
8103     0.607476
Name: label, dtype: float32

 # Random Forest feature importance
    -TEST THIS

In [169]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np
#Load boston housing dataset as an example

names = X.columns
rf = RandomForestRegressor()
rf.fit(X, y)
print ("Features sorted by their score:")
zipped = zip(map(lambda x: round(x, 4), rf.feature_importances_), names)
feature = sorted(zipped, key=lambda x: x[1])

/home/amaherok/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Features sorted by their score:


In [170]:
X.shape

(19414, 55)

In [171]:
feat_importance = pd.DataFrame(feature, columns=['importance', 'feature'])
feat_importance

,importance,feature
0,0.0129,COUNT(session)
1,0.0196,MAX(session.CHARACTERS(skus))
2,0.0000,MAX(session.NUMWORDS(skus))
3,0.0805,MAX(session.session)
4,0.0259,MAX(session.sku)
5,0.0203,MEAN(session.CHARACTERS(skus))
6,0.0000,MEAN(session.NUMWORDS(skus))
7,0.0379,MEAN(session.session)
8,0.0340,MEAN(session.sku)
9,0.0000,MIN(session.CHARACTERS(skus))


In [172]:
feat_importance =feat_importance.loc[feat_importance['importance'].isin(feat_importance['importance'].nlargest(100))]
feats_servibles = feat_importance['feature'].tolist()
feats_servibles

['COUNT(session)',
 'MAX(session.CHARACTERS(skus))',
 'MAX(session.NUMWORDS(skus))',
 'MAX(session.session)',
 'MAX(session.sku)',
 'MEAN(session.CHARACTERS(skus))',
 'MEAN(session.NUMWORDS(skus))',
 'MEAN(session.session)',
 'MEAN(session.sku)',
 'MIN(session.CHARACTERS(skus))',
 'MIN(session.NUMWORDS(skus))',
 'MIN(session.session)',
 'MIN(session.sku)',
 'MODE(session.DAY(timestamp))',
 'MODE(session.MONTH(timestamp))',
 'MODE(session.WEEKDAY(timestamp))',
 'MODE(session.YEAR(timestamp))',
 'NUM_UNIQUE(session.DAY(timestamp))',
 'NUM_UNIQUE(session.MONTH(timestamp))',
 'NUM_UNIQUE(session.WEEKDAY(timestamp))',
 'NUM_UNIQUE(session.YEAR(timestamp))',
 'NUM_UNIQUE(session.browser_version)',
 'NUM_UNIQUE(session.campaign_source)',
 'NUM_UNIQUE(session.channel)',
 'NUM_UNIQUE(session.city)',
 'NUM_UNIQUE(session.color)',
 'NUM_UNIQUE(session.condition)',
 'NUM_UNIQUE(session.country)',
 'NUM_UNIQUE(session.device_type)',
 'NUM_UNIQUE(session.event)',
 'NUM_UNIQUE(session.model)',
 'NUM_

In [173]:
df_top = df_top[feats_servibles]
df_top['person'] = subjects
df_top.head()

NameError: name 'df_top' is not defined

In [ ]:
feats_servibles

In [146]:
df_v = df.loc[df['event'] == 'viewed product']
df_v['timestamp'] = pd.to_datetime(df_v['timestamp'])
df_v['count'] = 1
df_v['month'] = df_v['timestamp'].dt.month
df_v['day'] = df_v['timestamp'].dt.day

df_v = df_v.groupby(['person','month', 'day','model']).agg({'count':'sum'})
#df_v = df_v.sort_values(by=['person','count'],ascending=False ).groupby('person').head(2).reset_index()

df_v = df_v.groupby(['person']).agg({'count':'mean'}).reset_index()
df_v.head()

,person,count
0,00091926,4.133333
1,00091a7a,1.000000
2,000ba417,4.371429
3,000c79fe,3.000000
4,000e4d9e,4.985294


In [147]:
df_s = df.loc[df['event'] == 'searched products']
df_s['timestamp'] = pd.to_datetime(df_s['timestamp'])
df_s['count'] = 1
df_s['month'] = df_s['timestamp'].dt.month
df_s['day'] = df_s['timestamp'].dt.day

df_s = df_s.groupby(['person','month', 'day', 'search_term']).agg({'count':'sum'})
#df_s = df_s.sort_values(by=['person','count'], ascending=False).groupby('person').head(2).reset_index()
df_s = df_s.groupby(['person']).agg({'count':'mean'}).reset_index()
df_s.head()

,person,count
0,000c79fe,1.142857
1,000e619d,3.000000
2,001001be,8.000000
3,001802e4,2.000000
4,0019e639,2.200000


In [148]:
df_s.columns = ['person','searched']
df_v.columns = ['person', 'viewed']
df_s.head()

,person,searched
0,000c79fe,1.142857
1,000e619d,3.000000
2,001001be,8.000000
3,001802e4,2.000000
4,0019e639,2.200000


In [149]:
df_vs_feat = df_v.merge(df_s,  on='person', how='left')
df_vs_feat.head()

,person,viewed,searched
0,00091926,4.133333,NaN
1,00091a7a,1.000000,NaN
2,000ba417,4.371429,NaN
3,000c79fe,3.000000,1.142857
4,000e4d9e,4.985294,NaN


In [270]:
df_top = df_top.merge(df_vs_feat, left_on='person', right_on='person', how='left')
df_top.shape

(38829, 3)